In [69]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import sympy as sym

import payoffs
import selection_functions

In [46]:
%matplotlib inline
plt.style.use("ggplot")

# Monomorphic gamma model

In [3]:
class U(sym.Function):
    """Generic matching function"""
    
    is_real = True
    
    @classmethod
    def eval(cls, x):
        """We require the U(0)=0 and U(1)=1"""
        if x.is_Number and x is sym.S.Zero:
            return sym.S.Zero
        elif x.is_Number and x is sym.S.One:
            return sym.S.One

        
class UGA(U):
    """Matching function for G males."""
    

class UgA(U):
    """Matching function for g males."""


In [4]:
x0, x1, x2 = sym.symbols('x0, x1, x2')
T, R, P, S = sym.symbols('T, R, P, S')

In [5]:
def N(x0, x1, x2, UGA, UgA, T, R, P, S):
    out = (
           2 * R *((x0 + x1) * UGA(x0 + x2)**2 + (1 - (x0 + x1)) * UgA(x0 + x2)**2) +
           2 * P * ((x0 + x1) * (1 - UGA(x0 + x2))**2 + (1 - (x0 + x1)) * (1 - UgA(x0 + x2))**2) +
           2 * (S + T) * ((x0 + x1) * UGA(x0 + x2) * (1 - UGA(x0 + x2)) + (1 - (x0 + x1)) * UgA(x0 + x2) * (1 - UgA(x0 + x2)))
          )
    return out

In [6]:
def equation_motion_GA_share(x0, x1, x2, UGA, UgA, T, R, P, S):
    numerator = (
                 x0 * UGA(x0 + x2)**2 * (1) * x0 / (x0 + x2) * 2*R +  # 2*R
                 x0 * UGA(x0 + x2)**2 * (1/2) * x2 / (x0 + x2) * 2*R + # 0

                 x0 * (1 - UGA(x0 + x2))**2 * (1/2) * x1 / (1 - x0 - x2) * 2*P + # 0
                 x0 * (1 - UGA(x0 + x2))**2 * (1/4) * (1 - x0 - x1 - x2) / (1 - x0 - x2) * 2*P + # 0

                 x0 * 2 * UGA(x0 + x2) * (1 - UGA(x0 + x2)) * (1) * x0 / (x0 + x2) * S + # 0
                 x0 * 2 * UGA(x0 + x2) * (1 - UGA(x0 + x2)) * (1/2) * x2 / (x0 + x2) * S + # 0
                 x0 * 2 * UGA(x0 + x2) * (1 - UGA(x0 + x2)) * (1/2) * x1 / (1 - x0 - x2) * T + # 0
                 x0 * 2 * UGA(x0 + x2) * (1 - UGA(x0 + x2)) * (1/4) * (1 - x0 - x1 - x2) / (1 - x0 - x2) * T + # 0

                 x1 * UGA(x0 + x2)**2 * (1/2) * x0 / (x0 + x2) * 2*R + # 0
                 x1 * UGA(x0 + x2)**2 * (1/4) * x2 / (x0 + x2) * 2*R + # 0

                 x1 * (1 - UGA(x0 + x2))**2 * (0) + # 0

                 x1 * 2 * UGA(x0 + x2) * (1 - UGA(x0 + x2)) * (1/2) * x0 / (x0 + x2) * S + # 0
                 x1 * 2 * UGA(x0 + x2) * (1 - UGA(x0 + x2)) * (1/4) * x2 / (x0 + x2) * S + # 0

                 x2 * UgA(x0 + x2)**2 * (1/2) * x0 / (x0 + x2) * 2*R + # 0

                 x2 * (1 - UgA(x0 + x2))**2 * (1/4) * x1 / (1 - x0 - x2) * 2* P + # 0

                 x2 * 2 * UgA(x0 + x2) * (1 - UgA(x0 + x2)) * (1/2) * x0 / (x0 + x2) * S + # 0
                 x2 * 2 * UgA(x0 + x2) * (1 - UgA(x0 + x2)) * (1/4) * x1 / (1 - x0 - x2) * T + # 0

                 (1 - x0 - x1 - x2) * UgA(x0 + x2)**2 * (1/4) * x0 / (x0 + x2) * 2*R + # 0

                 (1 - x0 - x1 - x2) * (1 - UgA(x0 + x2))**2 * (0) + # 0

                 (1 - x0 - x1 - x2) * 2 * UgA(x0 + x2) * (1 - UgA(x0 + x2)) * (1/4) * x0 / (x0 + x2) * S # 0
                 )

    x0_dot = (numerator / N(x0, x1, x2, UGA, UgA, T, R, P, S)) - x0
    return x0_dot


In [63]:
equation_motion_GA_share(x0, 1-x0, 0, UGA, UgA, T, R, P, S)

-x0 + (1.0*x0*(-UGA(x0) + 1)**2 + 9.0*x0*(-UGA(x0) + 1)*UGA(x0) + 10*x0*UGA(x0)**2 + 5.0*(-x0 + 1)*UGA(x0)**2)/(2*(-UGA(x0) + 1)**2 + 18*(-UGA(x0) + 1)*UGA(x0) + 10*UGA(x0)**2)

## Derivation of the locus of potential equilibria
After setting the equation of motion equal to zero, rearranging the resulting equation yields the locus of potential equilibria.

\begin{align}
 % 0 =& -x0 + \frac{Px_0\big(-U_{GA}(x0) + 1\big)^2 + 2Rx_0U_{GA}(x_0)^2 + R(-x_0 + 1)U_{GA}(x_0)^2 + 2Sx_0\big(-U_{GA}(x_0) + 1\big)U_{GA}(x_0) + \frac{1}{2}S(-2x_0 + 2)\big(-U_{GA}(x_0) + 1\big)U_{GA}(x_0) + Tx_0\big(-U_{GA}(x_0) + 1)U_{GA}(x_0)}{2P(-U_{GA}(x_0) + 1)^2 + 2RU_{GA}(x_0)^2 + (2S + 2T)(-U_{GA}(x_0) + 1)U_{GA}(x_0)} \\
 % 0 =& -x0 + \frac{Px_0\big(1-U_{GA}(x0)\big)^2 + 2Rx_0U_{GA}(x_0)^2 + R(1-x_0)U_{GA}(x_0)^2 + 2Sx_0\big(1-U_{GA}(x_0)\big)U_{GA}(x_0) + S(1-x_0)\big(1-U_{GA}(x_0)\big)U_{GA}(x_0) + Tx_0\big(1-U_{GA}(x_0)\big)U_{GA}(x_0)}{2P(1-U_{GA}(x_0))^2 + 2RU_{GA}(x_0)^2 + 2(S + T)(1-U_{GA}(x_0))U_{GA}(x_0)} \\
 % 0 =& -x0 + \frac{Px_0 - 2Px_0U_{GA}(x0) + Px_0U_{GA}(x0)^2 + 2Rx_0U_{GA}(x_0)^2 + RU_{GA}(x_0) - Rx_0U_{GA}(x_0)^2 + 2Sx_0U_{GA}(x0)-2Sx_0U_{GA}(x_0)^2 + SU_{GA}(x_0) - SU_{GA}(x_0)^2 - Sx_0U_{GA}(x_0) + Sx_0U_{GA}(x_0)^2 + Tx_0U_{GA}(x_0) - Tx_0U_{GA}(x_0)^2}{2P(1-U_{GA}(x_0))^2 + 2RU_{GA}(x_0)^2 + 2(S + T)(1-U_{GA}(x_0))U_{GA}(x_0)} \\
 % 0 =& -x0 + \frac{Px_0 + \big(2Sx_0U_{GA}(x0) - 2Px_0U_{GA}(x0) - Sx_0U_{GA}(x_0) + Tx_0U_{GA}(x_0)\big) + \big(Px_0U_{GA}(x0)^2 + 2Rx_0U_{GA}(x_0)^2 - Rx_0U_{GA}(x_0)^2 - 2Sx_0U_{GA}(x_0)^2 + Sx_0U_{GA}(x_0)^2 - Tx_0U_{GA}(x_0)^2\big) + \big(RU_{GA}(x_0) + SU_{GA}(x_0)\big) - SU_{GA}(x_0)^2}{2P(1-U_{GA}(x_0))^2 + 2RU_{GA}(x_0)^2 + 2(S + T)(1-U_{GA}(x_0))U_{GA}(x_0)} \\
 % 0 =& -x0 + \frac{Px_0 + \big(2S - 2P - S + T\big)x_0U_{GA}(x0) + \big(P + 2R - R - 2S + S - T\big)x_0U_{GA}(x0)^2 + \big(R + S\big)U_{GA}(x_0) - SU_{GA}(x_0)^2}{2P(1-U_{GA}(x_0))^2 + 2RU_{GA}(x_0)^2 + 2(S + T)(1-U_{GA}(x_0))U_{GA}(x_0)} \\
 % 0 =& -x0 + \frac{Px_0 + \big((T + S) - 2P\big)x_0U_{GA}(x0) + \big((P + R) - (T + S)\big)x_0U_{GA}(x0)^2 + \big(R + S\big)U_{GA}(x_0) - SU_{GA}(x_0)^2}{2P(1-U_{GA}(x_0))^2 + 2RU_{GA}(x_0)^2 + 2(S + T)(1-U_{GA}(x_0))U_{GA}(x_0)} \\
 % 0 =& -x0 + \frac{Px_0 + \big((T + S) - 2P\big)x_0U_{GA}(x0) + \big((P + R) - (T + S)\big)x_0U_{GA}(x0)^2 + \big(R + S\big)U_{GA}(x_0) - SU_{GA}(x_0)^2}{2P\big(1 - 2U_{GA}(x_0) + U_{GA}(x_0)^2\big) + 2RU_{GA}(x_0)^2 + 2(S + T)\big(U_{GA}(x_0) - U_{GA}(x_0)^2\big)} \\
 % 0 =& -x0 + \frac{1}{2}\frac{Px_0 + \big((T + S) - 2P\big)x_0U_{GA}(x0) + \big((P + R) - (T + S)\big)x_0U_{GA}(x0)^2 + \big(R + S\big)U_{GA}(x_0) - SU_{GA}(x_0)^2}{P - 2PU_{GA}(x_0) + PU_{GA}(x_0)^2 + RU_{GA}(x_0)^2 + SU_{GA}(x_0) - SU_{GA}(x_0)^2 + TU_{GA}(x_0) - TU_{GA}(x_0)^2} \\
 % 0 =& -x0 + \frac{1}{2}\frac{Px_0 + \big((T + S) - 2P\big)x_0U_{GA}(x0) + \big((P + R) - (T + S)\big)x_0U_{GA}(x0)^2 + \big(R + S\big)U_{GA}(x_0) - SU_{GA}(x_0)^2}{P + \big(SU_{GA}(x_0)  + TU_{GA}(x_0) - 2PU_{GA}(x_0)\big) + \big(PU_{GA}(x_0)^2 + RU_{GA}(x_0)^2 - SU_{GA}(x_0)^2 - TU_{GA}(x_0)^2\big)} \\
 % 0 =& -x0 + \frac{1}{2}\frac{Px_0 + \big((T + S) - 2P\big)x_0U_{GA}(x0) + \big((P + R) - (T + S)\big)x_0U_{GA}(x0)^2 + \big(R + S\big)U_{GA}(x_0) - SU_{GA}(x_0)^2}{P + \big((T + S) - 2P\big)U_{GA}(x_0) + \big((P + R) - (T + S)\big)U_{GA}(x_0)^2} \\
 % 2x_0\bigg(P + \big((T + S) - 2P\big)U_{GA}(x_0) + \big((P + R) - (T + S)\big)U_{GA}(x_0)^2\bigg) =& Px_0 + \big((T + S) - 2P\big)x_0U_{GA}(x0) + \big((P + R) - (T + S)\big)x_0U_{GA}(x0)^2 + \big(R + S\big)U_{GA}(x_0) - SU_{GA}(x_0)^2 \\
 % 2Px_0 + 2\big((T + S) - 2P\big)x_0U_{GA}(x_0) + 2\big((P + R) - (T + S)\big)x_0U_{GA}(x_0)^2 =& Px_0 + \big((T + S) - 2P\big)x_0U_{GA}(x0) + \big((P + R) - (T + S)\big)x_0U_{GA}(x0)^2 + \big(R + S\big)U_{GA}(x_0) - SU_{GA}(x_0)^2 \\ 
 % Px_0 + \big((T + S) - 2P\big)x_0U_{GA}(x_0) + \big((P + R) - (T + S)\big)x_0U_{GA}(x_0)^2 =& \big(R + S\big)U_{GA}(x_0) - SU_{GA}(x_0)^2 \\ 
 % x_0\bigg(P + \big((T + S) - 2P\big)U_{GA}(x_0) + \big((P + R) - (T + S)\big)U_{GA}(x_0)^2\bigg) =& \big(R + S\big)U_{GA}(x_0) - SU_{GA}(x_0)^2 \\
 % x_0 =& \frac{\big(R + S\big)U_{GA}(x_0) - SU_{GA}(x_0)^2}{P + \big((T + S) - 2P\big)U_{GA}(x_0) + \big((P + R) - (T + S)\big)U_{GA}(x_0)^2} \\
  x^* =& \frac{\big(R + S\big)U_{GA}^* - SU_{GA}^{*2}}{P + \big((T + S) - 2P\big)U_{GA}^* + \big((P + R) - (T + S)\big)U_{GA}^{*2}} \\
\end{align}

We already know that $x^*=0, U_{GA}^*=0$ and $x^*=1, U_{GA}^*=1$ are potential equilibria. One can confirm that the above formula is correct by verifying that it works for these corner cases.

## Interactive plot of the locus of potential equilibria

In [66]:
def locus_of_potential_equilibria(U, T, R, P, S):
    numerator = (R + S) * U - S * U**2
    denominator = (P + ((T + S) - 2 * P) * U + ((P + R) - (T + S)) * U**2)
    return numerator / denominator

In [75]:
def plot_locus_of_potential_equilibria(d0, T, R, P, S):
    assert T > R > P > S
    us = np.linspace(0, 1, 100)
    xs = locus_of_potential_equilibria(us, T, R, P, S)

    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    ax.plot(us, xs, label="LPE")
    ax.set_ylabel(r"$x_0^*$", rotation="horizontal", fontsize=25)
    ax.set_xlabel(r"Equilibrium selection probability, $U_{GA}^*$", fontsize=25)
    ax.plot(us, us, 'k--')
    
    # add a kirkpatrick selection function for comparison
    kirkpatrick_us = selection_functions.kirkpatrick_selection(xs, d0)
    ax.plot(us, kirkpatrick_us, label="Kirkpatrick")
    ax.legend()
    
    plt.show()

In [76]:
# sliders used to control the Prisoner's Dilemma Payoffs
T_slider = widgets.FloatSlider(value=10, min=0, max=100, step=0.1, description=r"$T$")
R_slider = widgets.FloatSlider(value=8, min=0, max=100, step=0.1, description=r"$R$")
P_slider = widgets.FloatSlider(value=6, min=0, max=100, step=0.1, description=r"$P$")
S_slider = widgets.FloatSlider(value=4, min=0, max=100, step=0.1, description=r"$S$")

# slider that controls the parameters of the selection function
d0_slider = widgets.FloatSlider(value=1, min=1.0, max=10, step=0.05, description=r"$d_0$")

w = widgets.interactive(plot_locus_of_potential_equilibria, d0=d0_slider,
                        T=T_slider, R=R_slider, P=P_slider, S=S_slider)
display(w)

A Jupyter Widget